# Introduction to text analysis VII #

### Distances and similarities among strings ###

In this notebook we will study the techniques for studying the similarity of strings. To measure the similary of two strings there are several methods. The most famous one is the *Levenshtein distance*-

 The Levenshtein or edit distance is the number of characters that need to be substituted, inserted, or deleted, to transform s1 into s2. For example, transforming “rain” to “shine” requires three steps, consisting of two substitutions and one insertion: “rain” -> “sain” -> “shin” -> “shine”. These operations could have been done in other orders, but at least three steps are needed.

** edit distance **

In [1]:
from nltk.metrics.distance import edit_distance

In [2]:
s1 = "The pen is on the table"
s2 = "The cat is on the table"
s3 = "This is a more different string !"

d = edit_distance(s1,s2)

print(d)


d1 = edit_distance(s1,s3)

print(d1)

3
25


In [3]:
a = "mistake"
b = "mestake"
edit_distance(a,b)


1

** confronting sets with the jaccard distance **

In [7]:
from nltk.metrics.distance import jaccard_distance


tokens1 = set(s1.split())
tokens2 = set(s2.split())
tokens3 = set(s3.split())

print(tokens1)
print(tokens2)

d = jaccard_distance(tokens1,tokens3)

print(d)

{'is', 'pen', 'on', 'the', 'The', 'table'}
{'cat', 'is', 'on', 'the', 'The', 'table'}
0.9166666666666666


### Term Frequency, inverse document frequency ###

The frequency of a term computed with the *FreqDist* function is a first approximation of importance of a word in a text. However the most common terms such as *the* tends to be too much represented. To improve the evaluation we build the inverse document metric. 

Suppose we have a corpus of *D* documents

\begin{align}
D = {d_1, d_2, ... , d_n}
\end{align}

the frequency of each word is weighted against the importance of each document with the *inverse document frequency*

\begin{align}
idf(t,d) = log \frac{N_D}{N_{(t,D)}}
\end{align}

where $N_D$ is the number of documents, $N_{(t,D)}$ is the number of documents where the word $t$ appears. As a word can be absent from all documents a case similar makes a division by zero. 

If we weight the importance of a term for the number of times a word is appearing in the documents words such as *the* will tend to disappear (as they will appear in all documents), then the complete tf-idf formula can be expressed as

$$
tfidf(t,d,D) = tf(t,d) \cdot idf(t,D) 
$$

the product of the term frequency $tf(t,d)$ in the document $d$ per the frequency of the term $t$ in all the corpus $D$.The formula above has several versions with minor adjustements.

In python a high level library for machine learning and text analysis is **scikit-learn** here we leverage the functions for text analysis and in particular tf-idf

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import nltk

In [11]:
import csv
import codecs
import string


In [12]:
token_dict = {}
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

def removePunct(text):
    nt = []
    for t in text:
        if(t not in string.punctuation):
            nt.append(t)
        else:
            nt.append(" ")
    return "".join(nt)

In [13]:
csvin = "inaug_speeches.csv"

documents = []
with codecs.open(csvin,"r","iso-8859-2") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text = row['text']
        text = text.lower()
        
        no_punctuation = removePunct(text)
        documents.append(no_punctuation)

In [14]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(documents)

In [39]:
tfs

<58x5296 sparse matrix of type '<class 'numpy.float64'>'
	with 32347 stored elements in Compressed Sparse Row format>

Text it hasn't seen gets excluded, unless you call fit_and_transform(), which adds the document to the model, whereas transform does not.

In [15]:

#newstring = 'this sentence has unseen text such as computer but also king lord juliet'
newstring = "In this conflict of emotions all I dare aver is that it has been my faithful study to collect my duty"
#newstring = 'sentence'
newstring2 = "In this people some of emotions all I dare aver is that it has been my faithful study to collect my duty"
#n
response = tfidf.transform([newstring,newstring2])
print(response)


  (0, 4636)	0.127775908687
  (0, 4438)	0.383084428239
  (0, 2088)	0.127775908687
  (0, 1726)	0.148924666465
  (0, 1500)	0.443139569052
  (0, 1422)	0.159578827315
  (0, 1097)	0.354572058202
  (0, 906)	0.242708244759
  (0, 811)	0.27946708298
  (0, 368)	0.560219449939
  (1, 4636)	0.130508325523
  (1, 4438)	0.391276475956
  (1, 3329)	0.135009058615
  (1, 2088)	0.130508325523
  (1, 1726)	0.152109337738
  (1, 1500)	0.452615862598
  (1, 1422)	0.1629913319
  (1, 1097)	0.362154384723
  (1, 811)	0.285443331322
  (1, 368)	0.572199431707


** associating words and tf-idf frequencies **

In [57]:

feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print (feature_names[col], ' - ', response[0, col])

thi  -  0.127775908687
studi  -  0.383084428239
ha  -  0.127775908687
faith  -  0.148924666465
emot  -  0.443139569052
duti  -  0.159578827315
dare  -  0.354572058202
conflict  -  0.242708244759
collect  -  0.27946708298
aver  -  0.560219449939
